In [ ]:
from confluent_kafka import Consumer, KafkaError, KafkaException
import sys

conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'foo'
}

consumer = Consumer(conf)

running = True

topics = ["ejercicio3"]

def basic_consume_loop(consumer, topics):
    try:
        consumer.subscribe(topics)

        while running:
            msg = consumer.poll(timeout=1.0)
            if msg is None:
                continue

            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                else:
                    raise KafkaException(msg.error())
            else:
                print(msg)
                print('Mensaje Recibido: {}'.format(msg.value().decode('utf-8')))
    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

def shutdown():
    global running
    running = False

basic_consume_loop(consumer, topics)
